In [54]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

import json
import time
import random


chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
# 🚀 Use Undetected Chrome Driver
driver = uc.Chrome(version_main=133)

# Define job search URL
job_title = "Software Developer"
location = "Remote"
url = f"https://www.indeed.com/jobs?q={job_title}&l={location}"

# Open Indeed in Selenium
driver.get(url)
time.sleep(random.uniform(5, 8))  # Wait for page to load

# Simulate human scrolling
def human_scroll(driver):
    for _ in range(random.randint(3, 6)):  
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(random.uniform(1, 3))  
    time.sleep(random.uniform(2, 5))  

# Function to clean salary data
def clean_salary(salary):
    invalid_salaries = ["Full-time", "Part-time", "Contract", "Internship"]
    return "Not Provided" if salary in invalid_salaries else salary

# Function to extract job details
def extract_jobs():
    job_list = []
    
    jobs = driver.find_elements(By.CLASS_NAME, "resultContent")
    
    if not jobs:
        print("❌ No job postings found! Page might be blocking automation.")
    else:
        for job in jobs:
            try:
                title_tag = job.find_element(By.CLASS_NAME, "jobTitle")
                title = title_tag.text.strip() if title_tag else "N/A"
            except:
                title = "N/A"

            try:
                company_tag = job.find_element(By.CLASS_NAME, "css-1h7lukg")
                company = company_tag.text.strip() if company_tag else "N/A"
            except:
                company = "N/A"

            try:
                location_tag = job.find_element(By.CLASS_NAME, "css-1restlb")
                location = location_tag.text.strip() if location_tag else "N/A"
            except:
                location = "N/A"

            # 🔹 Salary Extraction
            try:
                salary_tag = job.find_element(By.XPATH, ".//span[contains(@class, 'js-match-insights-provider-4pmm6z')]")
                salary = salary_tag.text.strip() if salary_tag else "N/A"
            except:
                salary = "N/A"

            # 🔹 If salary is not found, try clicking to open job details
            if salary == "N/A":
                try:
                    job.click()
                    time.sleep(2)  # Allow the details to load
                    salary_detail = driver.find_element(By.XPATH, "//span[contains(@class, 'js-match-insights-provider-4pmm6z')]")
                    salary = salary_detail.text.strip() if salary_detail else "N/A"
                except:
                    salary = "N/A"

            # 🔹 Clean the salary data
            salary = clean_salary(salary)

            # Append extracted data
            job_list.append({
                "title": title,
                "company": company,
                "location": location,
                "salary": salary
            })
    
    return job_list

# Extract multiple pages
all_jobs = []
max_pages = 3  # 🔹 Modify this number to extract more pages

for page in range(max_pages):
    human_scroll(driver)
    all_jobs.extend(extract_jobs())

    # 🔹 Find and click the "Next" button
    try:
        next_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@aria-label, 'Next')]")))
        
        # Ensure the button is clickable
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)  # Small wait after scroll

        # Try JavaScript click if normal click fails
        try:
            next_button.click()
        except:
            driver.execute_script("arguments[0].click();", next_button)

        time.sleep(random.uniform(3, 6))  # Allow next page to load

    except:
        print("❌ No more pages or next button not found.")
        break

# Close browser
driver.quit()

# Save results to JSON
with open("job_postings.json", "w", encoding="utf-8") as f:
    json.dump(all_jobs, f, indent=4)

print(f"✅ Scraped {len(all_jobs)} job postings and saved to 'job_postings.json'.")

❌ No more pages or next button not found.
✅ Scraped 30 job postings and saved to 'job_postings.json'.


In [46]:
!pip install webdriver-manager
